# APPLIED DATA SCIENCE CAPSTONE PROJECT
#### This notebook will be used for the IBM Applied Data Science Capstone Project course

### WEEK 3 - Segmenting and Clustering Neighborhoods in Toronto

In [24]:
import pandas as pd
import numpy as np

#### QUESTION 1 - Create Toronto neighborood pandas Dataframe by scraping Wikipedia page

First of all, let's scrape the html Wikipedia page ad save it into a pandas dataframe

In [34]:
# Read the html table using pandas read_html
wiki_tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# There are only 2 tables in the page: we need the first one
wiki_table_df = wiki_tables[0]
wiki_table_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Now let's process the dataframe as required by the assignment

In [35]:
# First, delete the rows with 'Not assigned' Neightborohood
neigh_df = wiki_table_df[wiki_table_df['Borough'] != 'Not assigned'].reset_index(drop=True)
print('Toronto Boroughts:',neigh_df['Borough'].unique())

Toronto Boroughts: ['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'York' 'East Toronto' 'West Toronto' 'Central Toronto' 'Mississauga']


In [36]:
# Check if 'Not assigned' neighboroods exist
not_ass_neigh_df = neigh_df[neigh_df['Neighbourhood'] == 'Not assigned']
not_ass_neigh_df.head()

,Postal Code,Borough,Neighbourhood


In [39]:
# Display the obtained dataframe with its shape
neigh_df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [40]:
print('Shape of the dataframe:',neigh_df.shape)

Shape of the dataframe: (103, 3)


#### QUESTION 2 - Retrieve latitudes and longitudes using geocoder API 

#### QUESTION 3 - Cluster Toronto neighborood